<a href="https://colab.research.google.com/github/Vasundhhara/semantic-search-engine/blob/main/semantic_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [2]:
import openai

In [4]:
from getpass import getpass
openai.api_key = getpass()

··········


In [7]:
def get_embedding(messages: str, model="text-embedding-ada-002"):
   messages = messages.replace("\n", " ")
   return openai.Embedding.create(input = [messages], model=model)['data'][0]['embedding']

In [6]:
import pandas as pd

In [ ]:
df = pd.read_csv('output_file.csv')

In [ ]:
df = df.dropna()

In [ ]:
df = df.iloc[:-20]

print(df)

                                               messages
0                              We all are hustling here
1     Yeah I told you I will for sure.. Networking i...
2     Anyways..... please let me know if you get to ...
3                                          Will do that
4                                             Yeah okay
...                                                 ...
1365                                             Oh, OK
1366                         I'm doing this for myself 
1367              No it's not a part of any internship 
1368                                       That's great
1369  I'm interested in developing business relation...

[1307 rows x 1 columns]


In [ ]:
print(df)

                                               messages
0                              We all are hustling here
1     Yeah I told you I will for sure.. Networking i...
2     Anyways..... please let me know if you get to ...
3                                          Will do that
4                                             Yeah okay
...                                                 ...
1365                                             Oh, OK
1366                         I'm doing this for myself 
1367              No it's not a part of any internship 
1368                                       That's great
1369  I'm interested in developing business relation...

[1307 rows x 1 columns]


In [ ]:
import string
def contains_non_ascii(text):
    return any(char not in string.ascii_letters + string.digits + string.punctuation + " " for char in text)

# Apply the function to filter out messages with non-ASCII characters
filtered_df = df[~df['messages'].apply(contains_non_ascii)]

In [ ]:
print(filtered_df)

                                               messages
0                              We all are hustling here
1     Yeah I told you I will for sure.. Networking i...
2     Anyways..... please let me know if you get to ...
3                                          Will do that
4                                             Yeah okay
...                                                 ...
1365                                             Oh, OK
1366                         I'm doing this for myself 
1367              No it's not a part of any internship 
1368                                       That's great
1369  I'm interested in developing business relation...

[1193 rows x 1 columns]


In [ ]:
df=filtered_df

In [ ]:
csv_filename = 'filtered_data.csv'
df.to_csv(csv_filename, index=False)

In [8]:
embeddings = []


In [ ]:
csv_filename = 'filtered_data.csv'
df.to_csv(csv_filename, index=False)
print(f'DataFrame saved to {csv_filename}')

DataFrame saved to filtered_data.csv


In [ ]:
for msg in df['messages']:
    embeddings.append(get_embedding(msg))

In [ ]:
df['embeddings'] = embeddings

In [ ]:
df.to_csv('embedded_file.csv',index=False)

In [ ]:
df.head()

,messages,embeddings
0,We all are hustling here,"[-0.02959604375064373, -0.001954703126102686, ..."
1,Yeah I told you I will for sure.. Networking i...,"[-0.0188064593821764, -0.005029025487601757, 0..."
2,Anyways..... please let me know if you get to ...,"[-0.028831681236624718, -0.017356514930725098,..."
3,Will do that,"[-0.021943015977740288, -0.002296175342053175,..."
4,Yeah okay,"[-0.0006003823364153504, -0.013631971552968025..."


In [ ]:
df = pd.read_csv('embedded_file.csv')

In [ ]:
embeddings = df['embeddings']

In [ ]:
new_embeddings = []
# convert the string representation of the lists to a list of floats
for e in embeddings:
    new_embeddings.append([float(x) for x in e[1:-1].split(',')])

In [ ]:
df['embeddings'] = new_embeddings
print(df.head())

   Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   

                                            messages  \
0                           We all are hustling here   
1  Yeah I told you I will for sure.. Networking i...   
2  Anyways..... please let me know if you get to ...   
3                                       Will do that   
4                                          Yeah okay   

                                          embeddings  
0  [-0.02959604375064373, -0.001954703126102686, ...  
1  [-0.0188064593821764, -0.005029025487601757, 0...  
2  [-0.028831681236624718, -0.017356514930725098,...  
3  [-0.021940890699625015, -0.0023069444578140974...  
4  [-0.0006003823364153504, -0.013631971552968025...  


In [ ]:
print(len(df['embeddings'][0]))

1536


In [ ]:
df.to_csv("final_embeddings.csv")

In [ ]:
df.shape

(1193, 4)

In [ ]:
df = df.loc[:, ["messages", "embeddings"]]

In [ ]:
df.head()

,messages,embeddings
0,We all are hustling here,"[-0.02959604375064373, -0.001954703126102686, ..."
1,Yeah I told you I will for sure.. Networking i...,"[-0.0188064593821764, -0.005029025487601757, 0..."
2,Anyways..... please let me know if you get to ...,"[-0.028831681236624718, -0.017356514930725098,..."
3,Will do that,"[-0.021940890699625015, -0.0023069444578140974..."
4,Yeah okay,"[-0.0006003823364153504, -0.013631971552968025..."


In [9]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 9.5 MB/s eta 0:00:00


In [10]:
api_key=getpass()

··········


In [12]:
import pinecone
pinecone.init(api_key=api_key, environment='asia-southeast1-gcp-free')

In [13]:
index = pinecone.Index("messages")

In [ ]:
vals = []
for i in range(len(df)):
    vals.append((df['messages'][i], df['embeddings'][i]))

In [ ]:
print(vals[0])

('We all are hustling here', [-0.02959604375064373, -0.001954703126102686, 0.003958601970225573, -0.021947775036096573, -0.013013863004744053, 0.0019514234736561775, -0.00019606476416811347, 0.01538836769759655, -0.009714481420814991, -0.052842579782009125, 0.003340377938002348, -0.0070579214952886105, 0.0005169632495380938, -0.0003904896730091423, 0.005562376696616411, -0.018222032114863396, 0.0255948044359684, -0.002508973004296422, 0.0041357059963047504, -0.025516092777252197, -0.017067575827240944, 0.0054213497787714005, 0.009294679388403893, 0.0010987005662173033, -0.004837562330067158, -0.0003431389632169157, 0.007182550150901079, -0.01385346706956625, 0.014181436970829964, -0.01453564502298832, 0.01422079373151064, -0.003801176091656089, 0.0027549508959054947, -0.0017431622836738825, -0.025109408423304558, -0.03151138871908188, -0.007221906911581755, 0.0048736389726400375, 0.007602352183312178, -0.00023798347683623433, 0.01583440788090229, 0.008179579861462116, -0.01401089224964

In [ ]:
print(len(vals))

1193


In [ ]:
for i in range(0, len(vals), 100):
    try:
       batch = vals[i:i + 100]

        # Truncate IDs that are longer than 512 characters
       truncated_batch = [(entry[0][:512], entry[1]) for entry in batch]

        # Perform upsert with the truncated_batch
       index.upsert(truncated_batch)
     # index.upsert(vals[i:i+100])
    except Exception as e:
        print(f"Error: {e}")
    print(f"{i / len(vals) * 100:.2f}%")


0.00%
8.38%
16.76%
25.15%
33.53%
41.91%
50.29%
58.68%
67.06%
75.44%
83.82%
92.20%


In [ ]:
pinecone.whoami()

WhoAmIResponse(username='1369d0e', user_label='default', projectname='45fda4f')

In [16]:
query= 'hey ?'
query_vector= get_embedding(query);

In [17]:
print(query_vector)

[-0.014162718318402767, -0.006256283260881901, -0.02198469638824463, -0.038850028067827225, -0.025908678770065308, 0.029131023213267326, -0.010635032318532467, -0.005298026371747255, 0.005070643033832312, -0.008127321489155293, 0.01854146644473076, 0.0040604122914373875, -0.003498451318591833, 0.0021796294022351503, 0.0017167424084618688, 0.00211791112087667, 0.01426666509360075, -0.005954188760370016, 0.022101636976003647, -0.0207243449985981, 0.004141620825976133, 0.00302744354121387, 0.0043625072576105595, -0.021854763850569725, -0.008958893828094006, -0.009212263859808445, 0.02038651891052723, -0.01774887554347515, 0.016358589753508568, -0.03541978821158409, 0.01177194807678461, 0.017060229554772377, -0.036693133413791656, 0.0014958559768274426, -0.02334899641573429, -0.0069189430214464664, 0.0016436550067737699, -0.006704553030431271, 0.007536125369369984, -0.017216147854924202, 0.01198633760213852, -0.015527017414569855, -0.009192774072289467, -0.01607273705303669, -0.00915379356

In [ ]:
print(len(query_vector))

1536


In [19]:
num_results = 10
results = index.query(queries=[query_vector], top_k=num_results)

In [20]:
print(results)

{'results': [{'matches': [{'id': 'Hey !!', 'score': 0.940248609, 'values': []},
                          {'id': 'Hey !! ', 'score': 0.929555416, 'values': []},
                          {'id': 'Hey ', 'score': 0.92285347, 'values': []},
                          {'id': 'Hi ', 'score': 0.884514, 'values': []},
                          {'id': 'Hey Hi', 'score': 0.881471455, 'values': []},
                          {'id': 'Hey', 'score': 0.878693581, 'values': []},
                          {'id': 'Hiii', 'score': 0.871766, 'values': []},
                          {'id': "Hey ! Hope you're doing well ",
                           'score': 0.871497929,
                           'values': []},
                          {'id': 'How are you ?',
                           'score': 0.870068133,
                           'values': []},
                          {'id': 'Hi hello ',
                           'score': 0.868791401,
                           'values': []}],
              'names